In [1]:
#Import dependencies
import pandas as pd
import requests

#API pull for latest salary info from www.levels.fyi
salaryData = requests.get('https://www.levels.fyi/js/salaryData.json').json()
salary_df = pd.DataFrame(salaryData)
salary_df['level'] = salary_df.level.apply(lambda x: x.lower() )
salary_df['level'] = salary_df.level.apply(lambda x: x.strip() )

#dropping columns that are not relevant to project
salary_df = salary_df.drop(['cityid', 'dmaid','rowNumber','otherdetails','tag', 'basesalary', 'stockgrantvalue', 'bonus', 'gender'], axis=1)

#converting to float to allow for summary stats
salary_df["totalyearlycompensation"] = pd.to_numeric(salary_df["totalyearlycompensation"])
salary_df["yearsofexperience"] = pd.to_numeric(salary_df["yearsofexperience"])
salary_df["yearsatcompany"] = pd.to_numeric(salary_df["yearsatcompany"])

#coverting timestamp from object to datetime
salary_df['timestamp'] =  pd.to_datetime(salary_df['timestamp'], infer_datetime_format=True)

# Create separate cols for city, state and country
def split_location(location):
    items = location.split(', ')
    city = items[0]
    state = items[1]
    
    if len(items)==2:
        country = 'US'
    elif len(items)==3:
        country = items[2].strip()
    elif len(items)==4:
        country = ', '.join([i.strip() for i in items[2:]])
    else:
        country = None
        print(location)
        
    return [city, state, country]

salary_df['loc_items'] = salary_df.location.apply(lambda x: split_location(x))
salary_df['city'] = salary_df.loc_items.apply(lambda x: x[0])
salary_df['state'] = salary_df.loc_items.apply(lambda x: x[1])
salary_df['country'] = salary_df.loc_items.apply(lambda x: x[2])

# dropping location column  
salary_df = salary_df.drop(['location','loc_items'], axis=1)

#isolating US data for further exploration
us_df = salary_df[salary_df.country=='US'].copy()

#isolating us data to data scientist titles
us_df = us_df[us_df.title=='Data Scientist'].copy()

In [2]:
us_df

,timestamp,company,level,title,totalyearlycompensation,yearsofexperience,yearsatcompany,city,state,country
745,2018-06-05 14:06:30,LinkedIn,senior,Data Scientist,233.0,4.0,0.0,San Francisco,CA,US
772,2018-06-08 00:29:47,Amazon,l4,Data Scientist,140.0,2.0,2.0,Seattle,WA,US
776,2018-06-08 09:49:25,Microsoft,64,Data Scientist,218.0,11.0,11.0,Seattle,WA,US
782,2018-06-08 17:55:09,ebay,26,Data Scientist,180.0,10.0,5.0,San Jose,CA,US
796,2018-06-10 19:39:35,Twitter,staff,Data Scientist,500.0,4.0,4.0,San Francisco,CA,US
...,...,...,...,...,...,...,...,...,...,...
62569,2021-08-16 16:17:19,IBM,l5,Data Scientist,145.0,6.0,5.0,New City,NY,US
62578,2021-08-16 17:08:58,Booz Allen Hamilton,senior consultant,Data Scientist,110.0,0.0,0.0,West McLean,VA,US
62600,2021-08-16 21:02:37,Xandr,l1,Data Scientist,120.0,1.0,0.0,Portland,OR,US
62610,2021-08-16 22:19:48,Facebook,l4,Data Scientist,233.0,2.0,2.0,Menlo Park,CA,US


In [3]:
#normalize formating inconsistency in levels
us_df.level.replace('senior associate', '4', inplace=True)
us_df.level.replace('principal associate', '5', inplace=True)
us_df.level.replace('manager', '5', inplace=True)
us_df.level.replace('director', '7', inplace=True)
us_df.level.replace('senior manager', '6', inplace=True)
us_df.level.replace('senior', '5', inplace=True)
us_df.level.replace('senior software engineer', '6', inplace=True)
us_df.level.replace('senior data scientist', '5', inplace=True)
us_df.level.replace('data scientist', '4', inplace=True)
us_df.level.replace('associate', '4', inplace=True)
us_df.level.replace('senior consultant', '5', inplace=True)
us_df.level.replace('lead associate', '5', inplace=True)
us_df.level.replace('staff', '4', inplace=True)
us_df.level.replace('staff software engineer', '6', inplace=True)
us_df.level.replace('group manager', '6', inplace=True)
us_df.level.replace('analyst', '4', inplace=True)
us_df.level.replace('vice president', '8', inplace=True)
us_df.level.replace('senior consultant', '5', inplace=True)
us_df.level.replace('consultant', '4', inplace=True)
us_df.level.replace('executive director / vice-president', '8', inplace=True)
us_df.level.replace('senior mts', '6', inplace=True)
us_df.level.replace('lead mts', '5', inplace=True)
us_df.level.replace('ii', '5', inplace=True)
us_df.level.replace('data scientist ii', '5', inplace=True)
us_df.level.replace('data scientist i', '4', inplace=True)
us_df.level.replace('only one level across netflix', '5', inplace=True)


In [4]:
co_df = us_df[us_df.company=='Capital One'].copy()
co_df.level.value_counts()

5                              20
4                              16
7                               2
6                               2
senior quantitative analyst     1
l4                              1
master software eng             1
l1                              1
Name: level, dtype: int64

In [5]:
nf_df = us_df[us_df.company=='Netflix'].copy()
nf_df.level.value_counts()

5                             26
6                              1
l6                             1
senior analytical engineer     1
data engineer                  1
Name: level, dtype: int64

In [6]:
ln_df = us_df[us_df.company=='LinkedIn'].copy()
ln_df.level.value_counts()

5                    12
6                     7
l3                    3
ic2                   3
l4                    2
4                     2
entry level           1
l1                    1
l2                    1
software engineer     1
Name: level, dtype: int64

In [7]:
us_df.level.value_counts()

l5                    231
l4                    195
5                     183
4                     144
l3                    114
                     ... 
qac1                    1
ic16                    1
officer                 1
mid senior              1
sr. data scientist      1
Name: level, Length: 323, dtype: int64

In [8]:
bh_df = us_df[us_df.company=='Booz Allen Hamilton'].copy()
bh_df.level.value_counts()

4                                  11
5                                   8
senior lead technologist            1
lead technologist                   1
associate senior data scientist     1
l1                                  1
l2                                  1
Name: level, dtype: int64

In [9]:
in_df = us_df[us_df.company=='Intuit'].copy()
in_df.level.value_counts()

5                       8
4                       6
6                       6
software engineer 2     2
l3                      1
staff data scientist    1
principal pm            1
l4                      1
staff data analyst      1
Name: level, dtype: int64

In [10]:
jp_df = us_df[us_df.company=='JPMorgan Chase'].copy()
jp_df.level.value_counts()

4                     13
603                    6
602                    4
8                      3
also                   1
vp                     1
601                    1
analyst (501)          1
executive director     1
Name: level, dtype: int64

In [11]:
dl_df = us_df[us_df.company=='Deloitte'].copy()
dl_df.level.value_counts()

5    11
4     7
Name: level, dtype: int64

In [12]:
gs_df = us_df[us_df.company=='Goldman Sachs'].copy()
gs_df.level.value_counts()

8    7
4    4
5    1
Name: level, dtype: int64

In [13]:
sf_df = us_df[us_df.company=='Salesforce'].copy()
sf_df.level.value_counts()

6                          4
5                          2
software engineer (swe)    1
architect                  1
principal swe              1
l7                         1
7                          1
4                          1
Name: level, dtype: int64

In [14]:
tr_df = us_df[us_df.company=='Twitter'].copy()
tr_df.level.value_counts()

5                          4
l6                         4
4                          2
ds 1                       1
data scientist 1           1
ds ii (l5)                 1
l5                         1
senior swe                 1
ds ii                      1
software engineer (swe)    1
ds 2                       1
sr. data scientist (l6)    1
swe ii                     1
l2                         1
data scientist 2           1
Name: level, dtype: int64

In [15]:
def clean_string(x):
    try:
        num_list = [i for i in x if i.isdigit()]
        if len(num_list)<1:
            return None
        else:
            return int(''.join(num_list))
    except:
        None


In [16]:
us_df.level = us_df.level.apply(lambda x: clean_string(x))

In [17]:
us_df

,timestamp,company,level,title,totalyearlycompensation,yearsofexperience,yearsatcompany,city,state,country
745,2018-06-05 14:06:30,LinkedIn,5.0,Data Scientist,233.0,4.0,0.0,San Francisco,CA,US
772,2018-06-08 00:29:47,Amazon,4.0,Data Scientist,140.0,2.0,2.0,Seattle,WA,US
776,2018-06-08 09:49:25,Microsoft,64.0,Data Scientist,218.0,11.0,11.0,Seattle,WA,US
782,2018-06-08 17:55:09,ebay,26.0,Data Scientist,180.0,10.0,5.0,San Jose,CA,US
796,2018-06-10 19:39:35,Twitter,4.0,Data Scientist,500.0,4.0,4.0,San Francisco,CA,US
...,...,...,...,...,...,...,...,...,...,...
62569,2021-08-16 16:17:19,IBM,5.0,Data Scientist,145.0,6.0,5.0,New City,NY,US
62578,2021-08-16 17:08:58,Booz Allen Hamilton,5.0,Data Scientist,110.0,0.0,0.0,West McLean,VA,US
62600,2021-08-16 21:02:37,Xandr,1.0,Data Scientist,120.0,1.0,0.0,Portland,OR,US
62610,2021-08-16 22:19:48,Facebook,4.0,Data Scientist,233.0,2.0,2.0,Menlo Park,CA,US


In [18]:
us_df[us_df.level.isna()].company.value_counts().head(10)

Spotify           6
Amazon            5
Expedia           5
Twitter           4
Wells Fargo       3
Tesla             3
IBM               3
Salesforce        3
JPMorgan Chase    3
Microsoft         3
Name: company, dtype: int64

In [19]:
us_df[us_df.level.isna()]

,timestamp,company,level,title,totalyearlycompensation,yearsofexperience,yearsatcompany,city,state,country
897,2018-06-25 08:45:29,Tesla,NaN,Data Scientist,168.0,8.0,3.0,Palo Alto,CA,US
1150,2018-08-03 15:28:13,Qualcomm,NaN,Data Scientist,220.0,6.0,6.0,San Diego,CA,US
1297,2018-08-15 06:10:00,Amazon,NaN,Data Scientist,200.0,5.0,1.0,Boston,MA,US
1301,2018-08-15 11:57:44,Booz Allen Hamilton,NaN,Data Scientist,120.0,1.0,0.0,Washington,DC,US
1431,2018-08-19 20:31:37,Illumina,NaN,Data Scientist,175.0,6.0,2.0,San Diego,CA,US
...,...,...,...,...,...,...,...,...,...,...
61327,2021-08-06 13:37:53,Nielsen,NaN,Data Scientist,152.0,8.0,4.0,Washington,DC,US
61709,2021-08-10 09:27:42,Spotify,NaN,Data Scientist,237.0,4.0,1.0,New York,NY,US
61950,2021-08-11 16:43:06,Booz Allen Hamilton,NaN,Data Scientist,215.0,9.0,9.0,Annapolis Junction,MD,US
62097,2021-08-12 19:30:32,Bank of America,NaN,Data Scientist,175.0,5.0,3.0,New York,NY,US


In [20]:
us_df.dropna(inplace=True)

In [22]:
us_df

,timestamp,company,level,title,totalyearlycompensation,yearsofexperience,yearsatcompany,city,state,country
745,2018-06-05 14:06:30,LinkedIn,5.0,Data Scientist,233.0,4.0,0.0,San Francisco,CA,US
772,2018-06-08 00:29:47,Amazon,4.0,Data Scientist,140.0,2.0,2.0,Seattle,WA,US
776,2018-06-08 09:49:25,Microsoft,64.0,Data Scientist,218.0,11.0,11.0,Seattle,WA,US
782,2018-06-08 17:55:09,ebay,26.0,Data Scientist,180.0,10.0,5.0,San Jose,CA,US
796,2018-06-10 19:39:35,Twitter,4.0,Data Scientist,500.0,4.0,4.0,San Francisco,CA,US
...,...,...,...,...,...,...,...,...,...,...
62494,2021-08-15 22:31:26,Adobe,3.0,Data Scientist,250.0,5.0,4.0,San Jose,CA,US
62569,2021-08-16 16:17:19,IBM,5.0,Data Scientist,145.0,6.0,5.0,New City,NY,US
62578,2021-08-16 17:08:58,Booz Allen Hamilton,5.0,Data Scientist,110.0,0.0,0.0,West McLean,VA,US
62600,2021-08-16 21:02:37,Xandr,1.0,Data Scientist,120.0,1.0,0.0,Portland,OR,US


In [23]:
us_df.to_csv('salary.csv', index=False)